In [43]:
pip install gensim

������ ������� ��࠭��: 1251
Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/24.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.4 MB ? eta -:--:--
   ---------------------------------------- 0.3/24.4 MB ? eta -:--:--
   ---------------------------------------- 0.3/24.4 MB ? eta -:--:--
   ---------------------------------------- 0.3/24.4 MB

In [158]:
import numpy as np
import pandas as pd
import re

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupShuffleSplit
from gensim.models import FastText
from sklearn.base import BaseEstimator, TransformerMixin
from collections import Counter
from sklearn.preprocessing import Normalizer


from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import PassiveAggressiveClassifier

from sklearn.metrics import accuracy_score

import joblib
import json

ПС я не стал оставлять в финальной работе различный просмотр статистик, графики, хэды и дт.

# Объединение таблиц

Загрузка таблиц

In [108]:
customers = pd.read_csv('customers.csv')
merchants = pd.read_csv('merchants.csv')
receipts = pd.read_csv('receipts.csv')
terminals = pd.read_csv('terminals.csv')
transactions = pd.read_csv('transactions.csv')

Обработка receipts - так касумма по транзакции уже имеется, цену отдельных товаров можно не брать (по моему предположению показатели из серии разброс в чеке, минимальный или макимальный элемент не должны сильно влиять на модель, но я могу и ошибаться Y(^_^)Y ), а названия обьединил в одну строчку для будующего TF-IDF

In [109]:
receipts = receipts.dropna(subset=["item_name"])

receipts_agg = (receipts.groupby("transaction_id").agg(
        receipt_text=("item_name", lambda x: " ".join(x.astype(str))), # эту конструкцию по запросу написал генеративный ии
        receipt_prices=("item_price", lambda x: list(x.astype(float)))
    ).reset_index()
)

In [110]:
receipts_agg

,transaction_id,receipt_text,receipt_prices
0,TX00000000,unit part,"[1.81, 48.01]"
1,TX00000001,thing,[9.87]
2,TX00000002,Ahsen Sernade item item,"[80.01, 78.5]"
3,TX00000003,Cacsae item Elsewhee,[36.83]
4,TX00000004,part aRivulet stuff dRiviulet Suafron piece,"[27.37, 64.59, 5.96]"
...,...,...,...
1427,TX00001495,Moonthreaed thing,[10.76]
1428,TX00001496,thing part,[45.38]
1429,TX00001497,thing object,"[17.17, 81.49]"
1430,TX00001498,Auorelic uEmbrace oonthread thing Gulow thing,"[62.41, 32.47, 37.63, 8.87]"


Обработка terminals - пропуски (их всего 2) замени на пустую строку, опять же во имя TF-IDF. обьеденил их с merchants и выбросил ненужные так как есть критерии прода(продакшена) столбцы.

In [111]:
merchants = merchants.drop(['merchant_name'], axis = 1)

In [112]:
terminals['terminal_description'] = terminals['terminal_description'].fillna("")
terminals_agg = (terminals.merge(merchants, on="merchant_id"))
terminals_agg = terminals_agg[['terminal_id', 'merchant_id', 'terminal_description', 'merchant_city']]

Далее 1 блока работы с transactions и последние 3 это объединение таблиц, замена пропусков на ничего во имя TF-IDF. ну и удаление бесполезных айдишников и парочки признаков которые я не доглядел. terminal_id используем в будущем для того что бы на train были другие терминалы в будующем.

In [113]:
transactions_agg = transactions[['transaction_id', 'terminal_id', 'customer_id', 'amount', 'item_count', 'true_mcc']]

In [114]:
data_f = (transactions_agg
    .merge(terminals_agg, on="terminal_id")
    .merge(customers, on="customer_id")
    .merge(receipts_agg, on="transaction_id")
)

In [115]:
data_f["receipt_text"] = data_f["receipt_text"].fillna("")

In [116]:
idshniki = ["transaction_id", "merchant_id", "customer_id", 'customer_segment']
data = data_f.drop(columns=idshniki)

Итог первого шага - обьединение всех таблиц и первое знакомство с данными. Заняло кстати у меня это +- 2 часа. Ибо я сразу не разобрался что на проде будут только эти данные и реально сидел отбирал что подойдет.

# Базовое решение, на основе которого будем строить оценку новых

Функция переводит все в строчные, убирает шумные символы if есть и убирает сдвоенные пробелы. Это все на всякий случай и для того что бы потом TF-IDF корректно работал.

In [117]:
def normalize_text(text):
    text = str(text).lower() 
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text


data['terminal_description'] = data['terminal_description'].apply(normalize_text)
data['merchant_city'] = data['merchant_city'].apply(normalize_text)
data['receipt_text'] = data['receipt_text'].apply(normalize_text)

In [118]:
data["full_text"] = (data["receipt_text"] + " " + data["terminal_description"] + " " + data["merchant_city"])
data["full_text"] = data["full_text"].apply(normalize_text)

Осевое решение

In [119]:
y = data['true_mcc']
x = data[['amount', 'item_count', 'full_text']]
groups = data['terminal_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, random_state= 456, stratify=y)

In [120]:
preprocessor = ColumnTransformer(
    transformers=[
        ("txt", TfidfVectorizer(
                ngram_range=(1, 2),
                min_df=2, # выбрано 2 для того что бы опечатки не были выбраны как признак, прописываю так как в задании был акцент на очепятки
                max_df=0.9,
                sublinear_tf=True), # что бы повторения не завышали веса
            "full_text"
        ),
        ("num", StandardScaler(),["amount", "item_count"]) # так как далее прменим линейку
    ]
)

model = LogisticRegression(max_iter=1000, class_weight="balanced")

pepa = Pipeline( steps=[("prep", preprocessor), ("clf", model)])


In [121]:
pepa.fit(x_train, y_train)

y_pred = pepa.predict(x_test)
acc = accuracy_score(y_test, y_pred)

print("Baseline accuracy:", round(acc, 4))

Baseline accuracy: 0.7909


# Улучшения

## Первое улучшение это попытка предотвратить leakage, так как на проде будут новые терминалы, то 

In [123]:
gss = GroupShuffleSplit(n_splits=1,test_size=0.2, random_state=11745)

train_id, test_id = next(gss.split(x, y, groups=data['terminal_id']))#теперь в тесте будут только новые терминалы

x_train = x.iloc[train_id]
x_test  = x.iloc[test_id]
y_train = y.iloc[train_id]
y_test  = y.iloc[test_id]

In [124]:
# analogichnо

model = LogisticRegression(max_iter=1000, class_weight="balanced")
pepa = Pipeline( steps=[("prep", preprocessor), ("clf", model)])

pepa.fit(x_train, y_train)
y_pred = pepa.predict(x_test)

acc = accuracy_score(y_test, y_pred)
print("Baseline accuracy:", round(acc, 4))

Baseline accuracy: 0.21


У меня если честно сердце здесь заболело, но это правдивая метрика, так как мы фактически еще и не решали

## Несколько статистик из товаров

In [125]:
def price_features(prices, amount):
    if not prices: return {
            "min": 0,
            "sum_ratio": 0
        }

    prices = np.array(prices)
    return {
        "min": prices.min(),
        "sum_ratio": prices.sum() / (amount + 0.00001) # xnj ,s yt gjltkbnm yf 0
    }


In [126]:
price_feats = data.apply(lambda row: price_features(row["receipt_prices"], row["amount"]), axis=1,result_type="expand")
data = pd.concat([data, price_feats], axis=1)

In [127]:
x = data.drop(['terminal_id', 'true_mcc', 'terminal_description', 'merchant_city', 'receipt_text', 'receipt_prices'], axis = 1) # меняем так как появились новые столбцы, остальное как было

x_train = x.iloc[train_id]
x_test  = x.iloc[test_id]
y_train = y.iloc[train_id]
y_test  = y.iloc[test_id]

In [128]:
preprocessor = ColumnTransformer(
    transformers=[
        ("txt", TfidfVectorizer(
                ngram_range=(1, 2),
                min_df=2,
                max_df=0.9,
                sublinear_tf=True),
            "full_text"
        ),
        ("num", StandardScaler(), ["amount", "item_count"]), 
        ("num_raw", "passthrough", ["min", "sum_ratio"]) # добавили новые признаки
    ]
)


model = LogisticRegression(max_iter=2000, class_weight="balanced")
pepa = Pipeline( steps=[("prep", preprocessor), ("clf", model)])

pepa.fit(x_train, y_train)
y_pred = pepa.predict(x_test)

acc = accuracy_score(y_test, y_pred)
print("Baseline accuracy:", round(acc, 4))

Baseline accuracy: 0.2135


## Борба с опечатками

In [129]:
# делю текст на описание товаров(врорде как реально важен) и доп информацию
data["context_text"] = (data["terminal_description"] + " " + data["merchant_city"]).apply(normalize_text)

In [130]:
x = data.drop(['terminal_id', 'true_mcc', 'terminal_description', 'merchant_city', 'receipt_prices', 'full_text'], axis = 1)# вновь новые колонки

x_train = x.iloc[train_id]
x_test  = x.iloc[test_id]
y_train = y.iloc[train_id]
y_test  = y.iloc[test_id]

In [131]:
#переконопатили всю архитектуру
preprocessor = ColumnTransformer(
    transformers=[
        ("receipt_txt", TfidfVectorizer(
                ngram_range=(1, 2),
                min_df=2,
                max_df=0.9,
                sublinear_tf=True
            ), "receipt_text"
        ),
        ("receipt_char", TfidfVectorizer( # эта часть призвана помочь с опечатками блягодоря analyzer="char" и ngram_range=(3, 5)
                analyzer="char",
                ngram_range=(3, 5),
                min_df=2,
                max_df=0.9,
                sublinear_tf=True
            ), "receipt_text"
        ),
        ("context_txt", TfidfVectorizer(
                ngram_range=(1, 2),
                min_df=2,
                max_df=0.9
            ), "context_text"
        ),
        ("num", StandardScaler(), ["amount", "item_count"]),
        ("num_raw", "passthrough", ["min", "sum_ratio"])
    ]
)


model = LogisticRegression(max_iter=2000, class_weight="balanced")
pepa = Pipeline( steps=[("prep", preprocessor), ("clf", model)])

pepa.fit(x_train, y_train)
y_pred = pepa.predict(x_test)

acc = accuracy_score(y_test, y_pred)
print("Baseline accuracy:", round(acc, 4))

Baseline accuracy: 0.4413


Как итог видим сильный буст, так как даже в задании прописано что опечаток будет много)))

## Замена алгоритма

естественно остановится на линейке мы не можем, протестируем еще парочку

Так как данные мы не меняем то preprocessor а так же все train/test таблицы берем из имеющихся уже

In [26]:
# Для скорости делал через нейронку
models = {
    "logreg": LogisticRegression(max_iter=3000, class_weight="balanced"),
    "sgd": SGDClassifier(loss="log_loss", alpha=1e-5, max_iter=3000, class_weight="balanced"),
    "linsvc": CalibratedClassifierCV(LinearSVC(class_weight="balanced"),  cv=10),
    "pa": PassiveAggressiveClassifier(max_iter=2000, class_weight="balanced")
}

for name, model in models.items():
    pipe = Pipeline([("prep", preprocessor), ("clf", model)])
    pipe.fit(x_train, y_train)
    acc = accuracy_score(y_test, pipe.predict(x_test))
    print(name, round(acc, 4))
    

logreg 0.4413
sgd 0.1032


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number 

linsvc 0.5374
pa 0.2847


Как итог лучшим показал себя LinearSVC(так как данных мало и самые главные в тексте), на нем и остановимся

## Реализация fastText

In [137]:
data['l2_rec'] = data['receipt_text'].apply(lambda x:len(x))

x = data.drop(['terminal_id', 'true_mcc', 'terminal_description', 'merchant_city', 'receipt_prices', 'full_text', 'l1_rec'], axis = 1)# вновь новые колонки

x_train = x.iloc[train_id]
x_test  = x.iloc[test_id]
y_train = y.iloc[train_id]
y_test  = y.iloc[test_id]

С помощью библиотеки обучаем модель векторизировать слова

In [138]:
cuprus = (x_train["receipt_text"].tolist() + x_train["context_text"].tolist())

model_fst = FastText(vector_size=100, window=5, min_count=1, sg=1, workers=4)

model_fst.build_vocab(cuprus)
model_fst.train(cuprus, total_examples=len(cuprus), epochs=10)

model_fst =  model_fst.wv

подсчет частот

In [139]:
word_freq = Counter()
for text in cuprus:
    word_freq.update(text.split())

Класс который я списал полностью, так как +- это было уже реализовано в статье которую я читал + GPT помог переделать под мою задачу

In [ ]:
class FastTextWeightedMean(BaseEstimator, TransformerMixin):
    def __init__(self, wv, dim, word_freq):
        self.wv = wv
        self.dim = dim
        self.word_freq = word_freq

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        n = len(X)
        result = np.zeros((n, self.dim), dtype=np.float32)

        for i, text in enumerate(X):
            tokens = str(text).split()
            weight_sum = 0

            for tok in tokens:
                if tok in self.wv:
                    w = 1 / (1 + self.word_freq.get(tok, 0))
                    result[i] += self.wv[tok][:self.dim] * w
                    weight_sum += w

            if weight_sum > 0:
                result[i] /= weight_sum

        return result
    
class FastTextWeightedMeanScaled(FastTextWeightedMean):
    def __init__(self, wv, dim, word_freq, scale=1.0):
        super().__init__(wv, dim, word_freq)
        self.scale = scale
    def transform(self, X):
        return super().transform(X) * self.scale

In [157]:
preprocessor = ColumnTransformer(
    transformers=[
        ("receipt_char", TfidfVectorizer(
                analyzer="char",
                ngram_range=(3, 5),
                min_df=2,
                max_df=0.9,
                sublinear_tf=True
            ), "receipt_text"
        ),
        ("ft_receipt", FastTextWeightedMeanScaled(model_fst, 50, word_freq, scale=1.5), "receipt_text"),
        ("ft_context", FastTextWeightedMeanScaled(model_fst, 50, word_freq, scale=0.7), "context_text"),
        ("num", StandardScaler(), ["amount", "item_count", "min", "sum_ratio",  "l2_rec"])
    ]
)

model = CalibratedClassifierCV(LinearSVC(class_weight="balanced"),  cv=10)
pepa = Pipeline( steps=[("prep", preprocessor), ("clf", model)])

pepa.fit(x_train, y_train)
y_pred = pepa.predict(x_test)

acc = accuracy_score(y_test, y_pred)
print("Baseline accuracy:", round(acc, 4))

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number 

Baseline accuracy: 0.7722


Как итог, интеграция FastText и удаление лишних параметров далли заметнейший буст, а так как дедлайн завтра, то это финальная версия.

# Финальная модель

In [ ]:
customers = pd.read_csv('customers.csv')
merchants = pd.read_csv('merchants.csv')
receipts = pd.read_csv('receipts.csv')
terminals = pd.read_csv('terminals.csv')
transactions = pd.read_csv('transactions.csv')
receipts = receipts.dropna(subset=["item_name"])

receipts_agg = (receipts.groupby("transaction_id").agg(
        receipt_text=("item_name", lambda x: " ".join(x.astype(str))),
        receipt_prices=("item_price", lambda x: list(x.astype(float)))
    ).reset_index()
)

merchants = merchants.drop(['merchant_name'], axis = 1)

terminals['terminal_description'] = terminals['terminal_description'].fillna("")
terminals_agg = (terminals.merge(merchants, on="merchant_id"))
terminals_agg = terminals_agg[['terminal_id', 'merchant_id', 'terminal_description', 'merchant_city']]
transactions_agg = transactions[['transaction_id', 'terminal_id', 'customer_id', 'amount', 'item_count', 'true_mcc']]

data_f = (transactions_agg
    .merge(terminals_agg, on="terminal_id")
    .merge(customers, on="customer_id")
    .merge(receipts_agg, on="transaction_id")
)

data_f["receipt_text"] = data_f["receipt_text"].fillna("")
idshniki = ["transaction_id", "merchant_id", "customer_id", 'customer_segment']
data = data_f.drop(columns=idshniki)

def normalize_text(text):
    text = str(text).lower() 
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

data['terminal_description'] = data['terminal_description'].apply(normalize_text)
data['merchant_city'] = data['merchant_city'].apply(normalize_text)
data['receipt_text'] = data['receipt_text'].apply(normalize_text)

def price_features(prices, amount):
    if not prices: return {
            "min": 0,
            "sum_ratio": 0
        }

    prices = np.array(prices)
    return {
        "min": prices.min(),
        "sum_ratio": prices.sum() / (amount + 0.00001)
    }

price_feats = data.apply(lambda row: price_features(row["receipt_prices"], row["amount"]), axis=1,result_type="expand")
data = pd.concat([data, price_feats], axis=1)

data["context_text"] = (data["terminal_description"] + " " + data["merchant_city"]).apply(normalize_text)
data['l2_rec'] = data['receipt_text'].apply(lambda x:len(x))

x = data.drop(['terminal_id', 'terminal_description', 'merchant_city', 'receipt_prices'], axis = 1)
y = data['true_mcc']



Как итог мы тайм лимит ловили с fast text

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("receipt_char", TfidfVectorizer(
                analyzer="char",
                ngram_range=(3, 5),
                min_df=2,
                max_df=0.9,
                sublinear_tf=True
            ), "receipt_text"
        ),
        ("num", StandardScaler(), ["amount", "item_count", "min", "sum_ratio",  "l2_rec"])
    ]
)

model = CalibratedClassifierCV(LinearSVC(class_weight="balanced"),  cv=10)
pepa = Pipeline( steps=[("prep", preprocessor), ("clf", model)])

pepa.fit(x, y)
joblib.dump(pepa, "model.pkl")


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number 

['model.pkl']